In [9]:
import numpy as np
import utils
from utils import loadmat
from joblib import Parallel, delayed
import multiprocessing

In [2]:
def pixelFeatures(x):
    return x.reshape(-1)

def feature_normalization(patch, type, epsilon=1e-5):
    patch = patch.astype('longdouble')
    if type == 'Sqrt':
        return np.sqrt(patch)
    elif type == 'L2-Norm':
        return patch / np.sqrt(np.sum(patch ** 2) + epsilon)
    else:
        return -1
    
def extract_relevant_window(arr, patch_size, index_i, index_j):
    curr_arr = np.zeros((patch_size, patch_size))
    for i in range(patch_size):
        for j in range(patch_size):
            curr_arr[i][j] = arr[index_j*patch_size + i][index_i*patch_size + j]     
    return curr_arr

def hogFeatures(x):
    # Applying Non Linear Mapping
    img = np.sqrt(x)

    # Computing the channel gradient
    r_grad, c_grad = np.empty(img.shape).astype('longdouble'), np.empty(img.shape).astype('longdouble')
    
    r_grad[1:-1,] = img[2:, :] - img[:-2, :] 
    c_grad[:, 1:-1] = img[:, 2:] - img[:, :-2]
    c_grad[:, 0], c_grad[:, -1] = 0, 0
    r_grad[0, :], r_grad[-1, 0] = 0, 0

    img_magnitude = np.sqrt(np.power(r_grad, 2) + np.power(c_grad, 2))
    img_theta = np.rad2deg(np.arctan(c_grad/(r_grad+0.00000001))) % 180
    orientation_bins = 8
    patch_size = 4
    tot_r, tot_c = img.shape
    hog = np.zeros((int(tot_r/patch_size), int(tot_c/patch_size), orientation_bins))
    for j in range(int(tot_c/patch_size)):
        for i in range(int(tot_r/patch_size)):
            # Extract the Current Patch and weight
            curr_patch = extract_relevant_window(img_theta, patch_size, i, j)
            curr_weight = extract_relevant_window(img_magnitude, patch_size, i, j)
            # Applying Histogram calculations
            hog[j][i] = np.histogram(np.ndarray.flatten(curr_patch), weights=np.ndarray.flatten(curr_weight), 
                                     bins=np.linspace(0, 180, num=(orientation_bins+1)))[0]
            
    hog_norm = feature_normalization(hog, 'L2-Norm')
    return hog_norm.ravel()

def compute_from_patch(patch):
    if patch.shape[0] != 3 and patch.shape[0] != patch.shape[1]:
        raise ValueError('Patch Size Mismatch')
    patch_sub = patch - np.ravel(patch)[4]
    patch_sub[patch_sub>0] = 1
    patch_sub[patch_sub<=0] = 0
    flattened_patch = np.delete(np.ravel(patch_sub), 4)
    return flattened_patch.dot(2**np.arange(flattened_patch.size)[::1])

def lbpFeatures(x):
    patch_size = 3
    img = x
    final_img = np.empty((img.shape[0]-(patch_size-1), img.shape[1]-(patch_size-1))).astype('longdouble')
    for r in range(0, final_img.shape[0]):
        for c in range(0, final_img.shape[1]):
            final_img[r][c] = compute_from_patch(img[r:r+patch_size, c:c+patch_size])
    return np.histogram(np.ndarray.flatten(final_img), bins=np.linspace(1, 255, num=257))[0]

In [3]:
data = utils.loadmat('../data/digits-normal.mat') 
N = data['x'].shape[2]

In [5]:
hog_features = np.empty((N, 392)).astype('longdouble')
for X_idx in range(N):
    hog_features[X_idx] = hogFeatures(data['x'][:, :, X_idx])

In [4]:
pixel_features = np.empty((N, 784)).astype('longdouble')
for X_idx in range(N):
    pixel_features[X_idx] = pixelFeatures(data['x'][:, :, X_idx])

In [6]:
lbp_features = np.empty((N, 256)).astype('longdouble')
for X_idx in range(N):
    lbp_features[X_idx] = lbpFeatures(data['x'][:, :, X_idx])

In [ ]:
start = time.time()
hog_features = np.empty((N, 392)).astype('longdouble')
for X_idx in range(N):
    hog_features[X_idx] = hogFeatures(data['x'][:, :, X_idx])
    
pixel_features = np.empty((N, 784)).astype('longdouble')
for X_idx in range(N):
    pixel_features[X_idx] = pixelFeatures(data['x'][:, :, X_idx])
    
lbp_features = np.empty((N, 256)).astype('longdouble')
for X_idx in range(N):
    lbp_features[X_idx] = lbpFeatures(data['x'][:, :, X_idx])

end = time.time()
print ("Time Taken = {}".format(end-start))

In [7]:
hog_features.shape, pixel_features.shape, lbp_features.shape

((2000, 392), (2000, 784), (2000, 256))

In [8]:
from joblib import Parallel, delayed
import multiprocessing

start1 = time.time()
N = data['x'].shape[2]
pixel_features2 = np.array(Parallel(n_jobs=multiprocessing.cpu_count()) 
                           (delayed(pixelFeatures)(data['x'][:, :, X_idx]) for X_idx in range(N)))

hog_features2 = np.array(Parallel(n_jobs=2) 
                           (delayed(hogFeatures)(data['x'][:, :, X_idx]) for X_idx in range(N)))

lbp_features2 = np.array(Parallel(n_jobs=multiprocessing.cpu_count()) 
                           (delayed(lbpFeatures)(data['x'][:, :, X_idx]) for X_idx in range(N)))

end1 = time.time()
print ("Time Taken = {}".format(end1-start1))

Time Taken = 22.422351837158203


In [10]:
hog_features2.shape, pixel_features2.shape, lbp_features2.shape

((2000, 392), (2000, 784), (2000, 256))

In [11]:
np.testing.assert_almost_equal(pixel_features2, pixel_features)

In [32]:
MAX = 1
np.testing.assert_almost_equal(hog_features2[:MAX], hog_features[:MAX])

AssertionError: 
Arrays are not almost equal to 7 decimals

Mismatch: 0.765%
Max absolute difference: 0.00078402
Max relative difference: 1.48633133e+198
 x: array([[0.0000000e+000, 0.0000000e+000, 0.0000000e+000, 0.0000000e+000,
        0.0000000e+000, 0.0000000e+000, 0.0000000e+000, 0.0000000e+000,
        0.0000000e+000, 0.0000000e+000, 0.0000000e+000, 0.0000000e+000,...
 y: array([[0.0000000e+000, 0.0000000e+000, 0.0000000e+000, 0.0000000e+000,
        0.0000000e+000, 0.0000000e+000, 0.0000000e+000, 0.0000000e+000,
        0.0000000e+000, 0.0000000e+000, 0.0000000e+000, 0.0000000e+000,...

In [12]:
np.testing.assert_almost_equal(lbp_features2, lbp_features)

In [33]:
hog_features2[1, :]

array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       9.03146362e-307, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       1.79775729e-307, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0

In [ ]:
# img = data['x'][:, :, 1]
# r_grad, c_grad = np.empty(img.shape).astype('longdouble'), np.empty(img.shape).astype('longdouble')

In [ ]:
# inputs = [data['x'][:, :, X_idx] for X_idx in range(N)]
# pixel_features = np.Parallel(n_jobs=multiprocessing.cpu_count())(delayed(pixelFeatures)(i) for i in inputs)
